# Introduction to [PowerSystems.jl](https://github.com/NREL-SIIP/PowerSystems.jl)

**Originally Contributed by**: Clayton Barrows and Jose Daniel Lara

## Introduction

This notebook is intended to show a power system data specification framework that exploits the
capabilities of Julia to improve performance and allow modelers to develop modular software
to create problems with different complexities and enable large scale analysis.

### Objective
PowerSystems.jl provides a type specification for bulk power system data.
The objective is to exploit Julia's integration of dynamic types to enable efficient data
handling and enable functional dispatch in modeling and analysis applications
As explained in Julia's documentation:

"Julia’s type system is dynamic, but gains some of the advantages of static type systems
by making it possible to indicate that certain values are of specific types. This can be
of great assistance in generating efficient code, but even more significantly, it allows
method dispatch on the types of function arguments to be deeply integrated with the language."

For more details on Julia types, refer to the [documentation](https://docs.julialang.org/en/v1/)


## Environment and packages

PowerSystems.jl relies on a framework for data handling established in
[InfrastructureSystems.jl](https://github.com/NREL-SIIP/InfrastructureSystems.jl).
Users of PowerSystems.jl should not need to interact directly with InfrastructureSystems.jl

The examples in this notebook depend upon Julia 1.2 and a specific set of package releases
as defined in the `Manifest.toml`.

In [1]:
using Pkg
Pkg.status()

using SIIPExamples;
using PowerSystems;
using D3TypeTrees;

Project SIIPExamples v0.0.1
    Status `~/Documents/repos/SIIPExamples.jl/Project.toml`
  [9961bab8] Cbc v0.6.7
  [41994980] D3TypeTrees v0.1.1
  [a93c6f00] DataFrames v0.20.2
  [2cd47ed4] InfrastructureSystems v0.6.2
  [b6b21f68] Ipopt v0.6.1
  [2535ab7d] JSON2 v0.3.1
  [4076af6c] JuMP v0.21.2
  [98b081ad] Literate v2.4.0
  [47be7bcc] ORCA v0.3.1
  [f0f68f2c] PlotlyJS v0.13.1
  [91a5bcdd] Plots v1.1.1
  [5f7eddb3] PowerGraphics v0.3.3 [`../PowerGraphics.jl`]
  [e690365d] PowerSimulations v0.4.4 [`../PowerSimulations.jl`]
  [bcd98974] PowerSystems v0.15.1
  [9e3dc215] TimeSeries v0.18.0
  [0f1e0344] WebIO v0.8.13
  [ade2ca70] Dates 
  [56ddb016] Logging 
  [44cfe95a] Pkg 
  [9a3f8284] Random 
  [10745b16] Statistics 
  [8dfed614] Test 


## Types in PowerSystems
PowerSystems.jl provides a type hierarchy for specifying power system data. Data that
describes infrastructure components is held in `struct`s. For example, a `Bus` is defined
as follows with fields for the parameters required to describe a bus (along with an
`internal` field used by InfrastructureSystems to improve the efficiency of handling data).

In [2]:
print_struct(Bus)

mutable struct PowerSystems.Bus
    number::Int64
    name::String
    bustype::Union{Nothing, PowerSystems.BusTypes.BusType}
    angle::Union{Nothing, Float64}
    voltage::Union{Nothing, Float64}
    voltagelimits::Union{Nothing, NamedTuple{(:min, :max),Tuple{Float64,Float64}}}
    basevoltage::Union{Nothing, Float64}
    area::Union{Nothing, PowerSystems.Area}
    load_zone::Union{Nothing, PowerSystems.LoadZone}
    ext::Dict{String,Any}
    internal::InfrastructureSystems.InfrastructureSystemsInternal
end


### Type Hierarchy
PowerSystems is intended to organize data containers by the behavior of the devices that
the data represents. To that end, a type hierarchy has been defined with several levels of
abstract types starting with `PowerSystemType`:
- `Component`: includes all elements of power system data
  - `PowerSystems.Topology`: includes non physical elements describing network connectivity
  - `Service`: includes descriptions of system requirements (other than energy balance)
  - `Device`: includes descriptions of all the physical devices in a power system
- `PowerSystems.TechnicalParams`: includes structs that hold data describing the technical or economic capabilities of `Device`some
- `System`: collects all of the `Component`s

*The following trees are made with [D3TypeTrees](https://github.com/claytonpbarrows/D3TypeTrees.jl),
nodes that represent Structs will show the Fields in the hoverover tooltip.*

In [3]:
TypeTree(PowerSystemType)

PowerSystems.PowerSystemType
├──PowerSystems.Component
│  ├──PowerSystems.Service
│  │  ├──PowerSystems.AGC (0 children)
│  │  ├──PowerSystems.Reserve (2 children)
│  │  └──PowerSystems.Transfer (0 children)
│  ├──PowerSystems.Device
│  │  ├──PowerSystems.Branch (2 children)
│  │  ├──PowerSystems.StaticInjection (4 children)
│  │  └──PowerSystems.DynamicInjection (2 children)
│  └──PowerSystems.Topology
│     ├──PowerSystems.Bus (0 children)
│     ├──PowerSystems.AggregationTopology (2 children)
│     └──PowerSystems.Arc (0 children)
├──PowerSystems.System
└──PowerSystems.DeviceParameter
   ├──PowerSystems.DynamicComponent
   │  ├──PowerSystems.DynamicInverterComponent (6 children)
   │  ├──PowerSystems.DynamicGeneratorComponent (5 children)
   │  └──PowerSystems.InverterComponent (0 children)
   ├──PowerSystems.OperationalCost
   │  ├──PowerSystems.TwoPartCost (0 children)
   │  └──PowerSystems.ThreePartCost (0 children)
   ├──PowerSystems.ActivePowerControl
   │  └──PowerSystems.VirtualInertia (0 children)
   └──PowerSystems.ReactivePowerControl
      └──PowerSystems.ReactivePowerDroop (0 children)

### `Forecasts`
Every `Component` has a `_forecasts::InfrastructureSystems.Forecasts` field (even composite types).
`Forecasts` are used to hold time series information that describes the temporally dependent
data of fields within the same struct. For example, the `ThermalStandard._forecasts` field can
describe other fields in the struct (`available`, `activepower`, `reactivepower`), while
the `ThermalStandard.tech._forecasts` holds data for `rating` and other `TechThermal` fields.

`Forecast`s themselves can take the form of the following:

In [4]:
TypeTree(Forecast)

InfrastructureSystems.Forecast
├──InfrastructureSystems.ScenarioBased
├──InfrastructureSystems.Probabilistic
└──InfrastructureSystems.Deterministic

In each case, the forecast contains fields for `label` and `data` to identify the `Component`
field that the forecast describes, and the time series `data`. For example:

In [5]:
print_struct(Deterministic)

mutable struct InfrastructureSystems.Deterministic
    label::String
    data::TimeSeries.TimeArray
end


Examples of how to create and add forecasts to system can be found in the
[Add Forecasts Example](../PowerSystems.jl Examples/add_forecasts.ipynb)

### System
The `System` object collects all of the individual components into a single struct along
with some metadata about the system itself (e.g. `basepower`)

In [6]:
print_struct(System)

 struct PowerSystems.System
    data::InfrastructureSystems.SystemData
    basepower::Float64
    frequency::Float64
    bus_numbers::Set{Int64}
    runchecks::Bool
    internal::InfrastructureSystems.InfrastructureSystemsInternal
end


## Basic example
PowerSystems contains a few basic data files (mostly for testing and demonstration).

In [7]:
BASE_DIR = abspath(joinpath(dirname(Base.find_package("PowerSystems")), ".."))
include(joinpath(BASE_DIR, "test", "data_5bus_pu.jl")) #.jl file containing 5-bus system data
nodes_5 = nodes5() # function to create 5-bus buses

5-element Array{PowerSystems.Bus,1}:
 PowerSystems.Bus(1, "nodeA", PowerSystems.BusTypes.PV, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("d10d5b50-94c4-4661-a2dc-161f97a8fd2b"), nothing)) 
 PowerSystems.Bus(2, "nodeB", PowerSystems.BusTypes.PQ, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("5c6ca7f7-acb8-4110-a942-8153ccae6b5a"), nothing)) 
 PowerSystems.Bus(3, "nodeC", PowerSystems.BusTypes.PV, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("aa3817b4-ea06-4885-a265-7720450381c1"), nothing)) 
 PowerSystems.Bus(4, "nodeD", PowerSystems.BusTypes.REF, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("4b9dc561-6433-42e1-b99a-66e5c5064

### Create a `System`

In [8]:
sys = System(
    nodes_5,
    vcat(thermal_generators5(nodes_5), renewable_generators5(nodes_5)),
    loads5(nodes_5),
    branches5(nodes_5),
    nothing,
    100.0,
    nothing,
    nothing,
)

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Bus,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,5
2,Line,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,6
3,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,3
4,RenewableDispatch,RenewableGen <: Generator <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,3
5,ThermalStandard,ThermalGen <: Generator <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,5


### Accessing `System` Data
PowerSystems provides functional interfaces to all data. The following examples outline
the intended approach to accessing data expressed using PowerSystems.

PowerSystems enforces unique `name` fields between components of a particular concrete type.
So, in order to retrieve a specific component, the user must specify the type of the component
along with the name and system

#### Accessing components

In [9]:
@show get_component(Bus, sys, "nodeA")
@show get_component(Line, sys, "1")

get_component(Bus, sys, "nodeA") = PowerSystems.Bus(1, "nodeA", PowerSystems.BusTypes.PV, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("d10d5b50-94c4-4661-a2dc-161f97a8fd2b"), nothing))
get_component(Line, sys, "1") = PowerSystems.Line("1", true, 0.0, 0.0, PowerSystems.Arc(PowerSystems.Bus(1, "nodeA", PowerSystems.BusTypes.PV, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("d10d5b50-94c4-4661-a2dc-161f97a8fd2b"), nothing)), PowerSystems.Bus(2, "nodeB", PowerSystems.BusTypes.PQ, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("5c6ca7f7-acb8-4110-a942-8153ccae6b5a"), nothing)), InfrastructureSystems.InfrastructureSystemsInternal(UUID("01831621-12c2-418e-bbb7-8a2c1aa96c90"), nothing)), 0.00281, 0.0281, (from = 0.00356, to 

1 (PowerSystems.Line):
   name: 1
   available: true
   activepower_flow: 0.0
   reactivepower_flow: 0.0
   arc: PowerSystems.Arc(PowerSystems.Bus(1, "nodeA", PowerSystems.BusTypes.PV, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("d10d5b50-94c4-4661-a2dc-161f97a8fd2b"), nothing)), PowerSystems.Bus(2, "nodeB", PowerSystems.BusTypes.PQ, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("5c6ca7f7-acb8-4110-a942-8153ccae6b5a"), nothing)), InfrastructureSystems.InfrastructureSystemsInternal(UUID("01831621-12c2-418e-bbb7-8a2c1aa96c90"), nothing))
   r: 0.00281
   x: 0.0281
   b: (from = 0.00356, to = 0.00356)
   rate: 2.0
   anglelimits: (min = -0.7, max = 0.7)
   services: PowerSystems.Service[]
   ext: Dict{String,Any}()
   forecasts: InfrastructureSystems.Forecasts(Dict{InfrastructureSystems.ForecastKey,Infrastru

Similarly, you can access all the components of a particular type: *note: the return type
of get_components is a `FlattenIteratorWrapper`, so call `collect` to get an `Array`

In [10]:
get_components(Bus, sys) |> collect

5-element Array{PowerSystems.Bus,1}:
 PowerSystems.Bus(1, "nodeA", PowerSystems.BusTypes.PV, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("d10d5b50-94c4-4661-a2dc-161f97a8fd2b"), nothing)) 
 PowerSystems.Bus(3, "nodeC", PowerSystems.BusTypes.PV, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("aa3817b4-ea06-4885-a265-7720450381c1"), nothing)) 
 PowerSystems.Bus(5, "nodeE", PowerSystems.BusTypes.PV, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("cf2f44f4-e49a-4a05-8099-ac0f6ab6c1ea"), nothing)) 
 PowerSystems.Bus(2, "nodeB", PowerSystems.BusTypes.PQ, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("5c6ca7f7-acb8-4110-a942-8153ccae6b

`get_components` also works on abstract types:

In [11]:
get_components(Branch, sys) |> collect

6-element Array{PowerSystems.Branch,1}:
 PowerSystems.Line("4", true, 0.0, 0.0, PowerSystems.Arc(PowerSystems.Bus(2, "nodeB", PowerSystems.BusTypes.PQ, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("5c6ca7f7-acb8-4110-a942-8153ccae6b5a"), nothing)), PowerSystems.Bus(3, "nodeC", PowerSystems.BusTypes.PV, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("aa3817b4-ea06-4885-a265-7720450381c1"), nothing)), InfrastructureSystems.InfrastructureSystemsInternal(UUID("1a4bc9ce-7cf9-46db-85fc-98d6d22da772"), nothing)), 0.00108, 0.0108, (from = 0.00926, to = 0.00926), 11.148, (min = -0.7, max = 0.7), PowerSystems.Service[], Dict{String,Any}(), InfrastructureSystems.Forecasts(Dict{InfrastructureSystems.ForecastKey,InfrastructureSystems.ForecastInternal}(), InfrastructureSystems.Hdf5TimeSeriesStorage("/var/folders/27/2jr8c

The fields within a component can be accessed using the `get_*` functions:

In [12]:
bus1 = get_component(Bus, sys, "nodeA")
@show get_name(bus1);
@show get_voltage(bus1);

get_name(bus1) = "nodeA"
get_voltage(bus1) = 1.0


#### Accessing `Forecast`s

First we need to add some forecasts to the `System`

In [13]:
loads = collect(get_components(PowerLoad, sys))
for (l, ts) in zip(loads, load_timeseries_DA[2])
    add_forecast!(sys, l, Deterministic("activepower", ts))
end

Now that the system has some forecasts, we can see all of them:

In [14]:
iterate_forecasts(sys) |> collect

3-element Array{Any,1}:
 InfrastructureSystems.Deterministic("activepower", 24×1 TimeSeries.TimeArray{Float64,1,Dates.DateTime,Array{Float64,1}} 2024-01-02T00:00:00 to 2024-01-02T23:00:00)
 InfrastructureSystems.Deterministic("activepower", 24×1 TimeSeries.TimeArray{Float64,1,Dates.DateTime,Array{Float64,1}} 2024-01-02T00:00:00 to 2024-01-02T23:00:00)
 InfrastructureSystems.Deterministic("activepower", 24×1 TimeSeries.TimeArray{Float64,1,Dates.DateTime,Array{Float64,1}} 2024-01-02T00:00:00 to 2024-01-02T23:00:00)

If we want to access a specific forecast for a specific component, we need to specify:
 - Forecast type
 - `component`
 - initial_time
 - label

We can find the unique set of initial times of all the forecasts in the system:

In [15]:
get_forecast_initial_times(sys)

1-element Array{Dates.DateTime,1}:
 2024-01-02T00:00:00

Or for a specific component:

In [16]:
@show initial_times = get_forecast_initial_times(Deterministic, loads[1]);

initial_times = get_forecast_initial_times(Deterministic, loads[1]) = Dates.DateTime[2024-01-02T00:00:00]


We can find the fields for which a component has a forecast:

In [17]:
@show labels = collect(get_forecast_keys(loads[1]))

labels = collect(get_forecast_keys(loads[1])) = InfrastructureSystems.ForecastKey[InfrastructureSystems.ForecastKey(InfrastructureSystems.DeterministicInternal, 2024-01-02T00:00:00, "activepower")]


1-element Array{InfrastructureSystems.ForecastKey,1}:
 InfrastructureSystems.ForecastKey(InfrastructureSystems.DeterministicInternal, 2024-01-02T00:00:00, "activepower")

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*